In [3]:
!pip install kaggle


In [4]:
from google.colab import files

# Upload kaggle.json file
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [6]:
!kaggle competitions download -c medical-specialty-classification

  0% 0.00/3.50M [00:00<?, ?B/s]
100% 3.50M/3.50M [00:00<00:00, 164MB/s]


In [8]:
import zipfile
import os

# Replace 'target_directory' with the directory where you downloaded the dataset
with zipfile.ZipFile('/content/medical-specialty-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/medical_data_NLP')

# List the contents of the extracted directory
os.listdir('/content/medical_data_NLP')


['Sample Submission.csv', 'Train.csv', 'test.csv']

In [9]:
import pandas as pd

# Replace 'file_path' with the path to your dataset file
train_df = pd.read_csv('/content/medical_data_NLP/Train.csv')

train_df.head()

,Id,description,medical_specialty,sample_name,transcription
0,1,Patient had a recurrent left arm pain after h...,Cardiovascular / Pulmonary,Angina - Consult,"HISTORY OF PRESENT ILLNESS: , The patient is a..."
1,2,The patient is an 84-year-old female presente...,Cardiovascular / Pulmonary,Pericardial Effusion,"REASON FOR CONSULTATION:, Pericardial effusio..."
2,3,Hand dermatitis.,Dermatology,Dermatitis - SOAP,"SUBJECTIVE:, This is a 29-year-old Vietnamese..."
3,4,Recurrent degenerative spondylolisthesis and ...,Surgery,Lumbar Re-exploration,"PREOPERATIVE DIAGNOSIS: , Recurrent degenerati..."
4,5,Chiropractic IME with old files review. Deta...,IME-QME-Work Comp etc.,Chiropractic IME - 2,"DATE OF INJURY : October 4, 2000,DATE OF EXAMI..."


In [27]:
(train_df['medical_specialty'].unique())
#len(train_df['medical_specialty'].unique())
train_df[['Id','medical_specialty']].groupby('medical_specialty').count().sort_values('Id',ascending=False)

,Id
medical_specialty,
Surgery,788
Consult - History and Phy.,310
Cardiovascular / Pulmonary,309
Orthopedic,294
Radiology,225
Gastroenterology,189
General Medicine,188
Neurology,166
SOAP / Chart / Progress Notes,130


35 different categories. For the base model, we will be using only the 'description' column for classification.